In [2]:
data_dir = '../data/'
now = datetime.datetime.now()

def __my_flatten_cols(self, how="_".join, reset_index=True):
    how = (lambda iter: list(iter)[-1]) if how == "last" else how
    self.columns = [how(filter(None, map(str, levels))) for levels in self.columns.values] \
                    if isinstance(self.columns, pd.MultiIndex) else self.columns
    return self.reset_index() if reset_index else self
pd.DataFrame.my_flatten_cols = __my_flatten_cols

In [3]:
#Delinquency data
delinquency_data = pd.read_csv(data_dir + 'debtbyage_20181205_REFINED.csv')
delinquency_data['Delinquent Date'] =  pd.to_datetime(delinquency_data['Delinquent Date'], 
                                                      format='%m/%d/%Y')
delinquency_data['year'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).year
delinquency_data['month'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).month

delinquency_data = delinquency_data[['Assess Case Nbr', 'Mine ID', 'Mine Status',
       'Violator ID', 'Violator Name', 'Violator Type', 'Controller Id',
       'Controller Name', 'Age Category', 'Delinquent Date', 'Delinquent Type',
       'Ending Balance', 'Current Operator ID', 'Current Operator Name','year','month']]

delinquency_data = delinquency_data.rename(index=str, columns={'Assess Case Nbr':'ass_case_nbr',
                                                              'Mine ID':'mine_id',
                                                              'Mine Status':'mine_status',
                                                              'Violator ID':'violator_id',
                                                              'Violator Name':'violator_nm',
                                                              'Violator Type':'violator_type',
                                                              'Controller Id':'curr_cntrl_id',
                                                              'Controller Name':'curr_cntrl_nm',
                                                              'Age Category':'age_cateogry',
                                                              'Delinquent Date':'delinquent_date',
                                                              'Delinquent Type':'delinquent_type',
                                                              'Ending Balance':'ending_balance',
                                                               'Current Operator ID':'curr_oprtr_id',
                                                               'Current Operator Name':'curr_oprtr_nm'})

earliest_dict = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min').to_dict()
earliest_series = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min')
earliest_df = pd.DataFrame({'mine_id':earliest_series.index, 'earliest_date':earliest_series.values})  
#Need to round earliest_date up to the next year so we can factor only years with full delinquency
earliest_df['earliest_year'] = earliest_df['earliest_date'].map(lambda x: x.year+1)

mine_del_summary = delinquency_data.groupby('mine_id').agg({'delinquent_date':['max','min'],
                                                            'ending_balance':['sum','count'], 
                                                            'curr_cntrl_id':'nunique',
                                                           'curr_cntrl_nm':'sum'}).my_flatten_cols()
mine_del_summary = mine_del_summary.rename(index=str, columns={'delinquent_date_max':'latest_del_date',
                                           'delinquent_date_min':'earliest_del_date','ending_balance_sum':'total_due',
                                           'ending_balance_count':'num_del_records','curr_cntrl_id_nunique':'num_cntrlrs',
                                           'curr_cntrl_nm_sum':'cntrlrs_concat'})

In [4]:
#Injury data with delinquency date and flag attached
accident_data = pd.read_csv(data_dir + 'msha_accident_20190302_y94_y19.csv', escapechar='\\')

injury_occurred = accident_data.no_injuries > 0
not_contractor = pd.isna(accident_data.cntctr_id)
not_natural = accident_data.degree_injury_cd.isin(['01','02','03','04','05','06','07','10'])

inj_data = accident_data.loc[(injury_occurred) & (not_contractor) & (not_natural)]

inj_data['year'] = pd.DatetimeIndex(inj_data['ai_dt']).year
inj_data['month'] = pd.DatetimeIndex(inj_data['ai_dt']).month
inj_data['yearmonth'] = inj_data['year'].map(str) + inj_data['month'].map(str)
inj_data['delinquent_date'] = inj_data['mine_id'].map(earliest_series)
inj_data['delinquent'] = np.where(pd.to_datetime(inj_data['ai_dt'], 
                                                 format='%Y/%m/%d') > inj_data['delinquent_date'],1,0)

/Users/akanik/.virtualenvs/simple-csvkit-p3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/akanik/.virtualenvs/simple-csvkit-p3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/akanik/.virtualenvs/simple-csvkit-p3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set

In [ ]:
#Violation data with delinquency date and flag attached
viol_data = pd.read_csv(data_dir + 'msha_violations_20190302_y94_y19.csv')
    
viol_data['delinquent_date'] = viol_data['mine_id'].map(earliest_series)
viol_data['delinquent'] = np.where(pd.to_datetime(viol_data['iss_dt'], 
                                                  format='%Y/%m/%d') > viol_data['delinquent_date'],1,0)

In [5]:
#Employee hours and production
hrs_data = pd.read_csv(data_dir + 'msha_cy_oprtr_emplymnt_20190302_y94_y19.csv')

In [6]:
#Controller-Operation relationship data
cntrlr_hist_data = pd.read_csv(data_dir + 'msha_controller_history_20190105-0.csv')
cntrlr_hist_data['ctrlr_end_dt'] = cntrlr_hist_data['ctrlr_end_dt'].fillna('2019-01-01')
cntrlr_hist_data['ctrlr_start_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_start_dt'], 
                                                      format='%Y-%m-%d')
cntrlr_hist_data['ctrlr_end_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_end_dt'], 
                                                      format='%Y-%m-%d')

In [7]:
#Mine-specific data
mine_data = pd.read_csv(data_dir + 'msha_mine_20190209-0.csv', escapechar='\\')
       
mine_rename = {'controller_id':'curr_cntrlr_id', 'controller_nm':'curr_cntrlr_nm', 
                'oper_id':'curr_oper_id', 'oper_nm':'curr_oper_nm'}
                
mine_data = mine_data.rename(index=str, columns=mine_rename)

abandoned = mine_data['curr_stat_cd'].isin(['Abandoned','AbandonedSealed'])
current_status = pd.to_datetime(mine_data['curr_stat_dt'], format='%d-%m-%Y') < pd.to_datetime('1994-01-01', format='%Y-%m-%d')

mine_data = mine_data[~((abandoned) & (current_status))]

Ok, now let's calculate the injury rate for a few different datasets. Here are the datasets and injury rates we'll want to calculate:
- Annual injury rate for all mines and all mine types (get_total_injury_rate)
- Annual delinquent or nondelinquent* injury rates for each mine (get_del_injury_rate)
- Annual injury rate for delinquent mines vs. nondelinquent mines
- Past 5-year aggregate injury rate for delinquent vs. nondelinquent mines
- Aggregate delinquent vs. nondelinquent injury rates for each mine

*Each year will be either delinquent, nondelinquent or becoming. I'd like to include the "becoming" category in there too. I think that adds good context

### Injury rate caveats: 
With the incomplete delinquency data that we have, the farther back in time you go the more likely you are to be missing data related to mine delinquencies. 

Injury rate is calculated by dividing annual hours logged at an individual mine by 2,000 to get a number that represents fulltime employees (FTE). If there were fewer than 2,000 hours logged at a mine in a year and that mine still had an accident, their injury rate goes off the charts. I think that the thing to do is to not calculate injury rates for mines in years where they had fewer than 2,000 hours logged. In other words, don't calculate injury rates for years in which the mine didn't even have a single fulltime employee working at the mine.

But we definitely still need to include their accident counts in totals and probably pull out mines that have fewer than 2,000 hours logged but that still have accidents so we can analyze those separately.

###########

Ok here are some helper functions:

In [13]:
def find_rate_type(row):
    if row['cal_yr'] >= row['earliest_year']:
        return 'Delinquent'
    elif row['cal_yr'] == (row['earliest_year']-1):
        return 'Became delinquent'
    else:
        return 'Non-delinquent'

def calc_injury_rate(row):    
    if row['annual_hrs'] < 2000:
        return 0
    else:
        return (row['inj_cnt'] / (row['annual_hrs'] / 2000)) * 100
    
def calc_del_injury_rate(row):
    if row['annual_hrs'] < 2000:
        return 0
    else:
        if row['rate_type'] == 'Delinquent':
            return (row['del_inj'] / (row['annual_hrs'] / 2000)) * 100
        elif row['rate_type'] == 'Became delinquent':
            return ((row['del_inj'] + row['non_inj']) / (row['annual_hrs'] / 2000)) * 100
        elif row['rate_type'] == 'Non-delinquent':
            return (row['non_inj'] / (row['annual_hrs'] / 2000)) * 100
        else:
            return -999999
        
def calc_joint_injury_rate(row):
    if row['annual_hrs'] < 2000:
        return 0
    else:
        return ((row['del_inj'] + row['non_inj']) / (row['annual_hrs'] / 2000)) * 100
    
    
#Create a unique list of currently delinquent mines
def get_delinquent_mines():
    return delinquency_data['mine_id'].unique()

#Create a unique list of mines that have been delinquent since the year passed to this function
def get_delinquent_since(year): 
    delinquent_since = earliest_df[earliest_df['earliest_year'] <= year]
    return delinquent_since['mine_id'].unique()

A common thing we're going to want to do is take mine-level aggregate data and combine it with delinquency and other general mine data. So let's create a function that takes in a mine-level dataset and joins it with delinquency and general mine data.

In [86]:
def combine_del_mine_data(mine_level_agg):
    
    keep_cols = ['mine_id','curr_mine_nm', 'c_m_ind', 'mine_type_cd', 'curr_stat_cd',
           'curr_stat_dt', 'curr_cntrlr_id', 'curr_cntrlr_nm', 'curr_oper_id',
           'curr_oper_nm', 'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt']
    mine_data_refined = mine_data[keep_cols]
    
    agg_mine = pd.merge(mine_level_agg, mine_data_refined, 
                        how='left', on='mine_id', suffixes=('_a','_m'))
    agg_mine_del = pd.merge(agg_mine, mine_del_summary, 
                            how='left', on='mine_id', suffixes=('_am','_d'))

    return agg_mine_del

Ok now let's start writing our injury rate functions. We're going to make them agile so that they will accept different types of injury and employment data. That way, we can run the overall rate for things like:
- all mines
- all coal mines
- all mines owned by a certain controller

All with the same function.

In [10]:
def get_total_injury_rate(inj_data, hrs_data, rate_type):

    inj_mine_year = inj_data.groupby('cal_yr')['document_no'].count().reset_index()
    inj_mine_year = inj_mine_year.rename(index=str, columns={'document_no': 'inj_cnt'})

    hrs_prod_mine_year = hrs_data.groupby('calendar_yr', as_index=False)['annual_hrs'].sum()
    hrs_prod_mine_year = hrs_prod_mine_year.rename(index=str, columns={'calendar_yr': 'cal_yr'})

    hrs_prod_inj_mine_year = pd.merge(hrs_prod_mine_year, inj_mine_year, 
                                      how='left', on=['cal_yr'], suffixes=('_m','_inj'))
    #hrs_prod_inj_mine_year = hrs_prod_inj_mine_year.rename(index=str, columns={0: 'non_inj',1:'del_inj'})

    hrs_prod_inj_mine_year['inj_rate'] = hrs_prod_inj_mine_year.apply(calc_injury_rate, axis=1)

    avg_inj_rate = hrs_prod_inj_mine_year[hrs_prod_inj_mine_year['cal_yr'] < 2019]['inj_rate'].mean()
    
    #return hrs_prod_inj_mine_year
    #return hrs_prod_inj_mine_year.groupby('inj_rate')['cal_yr'].count()
    return rate_type + ': ' + str(round(avg_inj_rate, 2))


And now let's test that function out:

In [158]:
del_inj_data = inj_data[inj_data['mine_id'].isin(get_delinquent_since(2014))]
del_hrs_data = hrs_data[hrs_data['mine_id'].isin(get_delinquent_since(2014))]

nondel_inj_data = inj_data[~inj_data['mine_id'].isin(get_delinquent_mines())]
nondel_hrs_data = hrs_data[~hrs_data['mine_id'].isin(get_delinquent_mines())]

print(get_total_injury_rate(inj_data, hrs_data, 'National'))
print(get_total_injury_rate(del_inj_data, del_hrs_data, 'Delinquent since 2014'))
print(get_total_injury_rate(nondel_inj_data, nondel_hrs_data, 'Not currently delinquent'))

National: 4.32
Delinquent since 2014: 5.56
Not currently delinquent: 4.15


---------------------
The above function is great, but it is returning all injury/employment data between 1994 and 2018 for mines that have been delinquent since 2014. Injury/employment data from years before they were delinquent are included there too. This is important to think about because we're calculating an average injury rate over as many years as are available for these mines. 

Instead, let's take the average 5-year injury rate for mines instead of just an average for all available years.

In [141]:
def get_range_injury_rate(inj_data, hrs_data, rate_type, year_list):

    inj_year = inj_data.groupby('cal_yr')['document_no'].count().reset_index()
    inj_year = inj_year.rename(index=str, columns={'document_no': 'inj_cnt'})

    hrs_year = hrs_data.groupby('calendar_yr', as_index=False)['annual_hrs'].sum()
    hrs_year = hrs_year.rename(index=str, columns={'calendar_yr': 'cal_yr'})

    hrs_inj_year = pd.merge(hrs_year, inj_year, 
                                      how='left', on=['cal_yr'], suffixes=('_m','_inj'))

    hrs_inj_year['inj_rate'] = hrs_inj_year.apply(calc_injury_rate, axis=1)

    avg_inj_rate = hrs_inj_year[hrs_inj_year['cal_yr'].isin(year_list)]['inj_rate'].mean()
    
    return rate_type + ': ' + str(round(avg_inj_rate, 2))


In [161]:
##5-yr average rates: Only group on year and calculate rate, then take mean
print(get_range_injury_rate(inj_data, hrs_data, 'National', list(range(2018,2019,1))))
print(get_range_injury_rate(del_inj_data, del_hrs_data, 'Delinquent since 2014', list(range(2018,2019,1))))
print(get_range_injury_rate(nondel_inj_data, nondel_hrs_data, 'Not currently delinquent', list(range(2018,2019,1))))

National: 2.29
Delinquent since 2014: 2.28
Not currently delinquent: 2.11


Not a huge change but I feel much better about this.

So at this point we can say that the 5-year average annual injury rate for all mines that reported employment data in the U.S. is 4.16 injuries per 100 workers. The 5-year average annual injury rate for mines that have been delinquent at least since 2014, so every year during the 5-year period we're calculating for, is 5.35. And the 5-year average annual injury rate for mines that are not currently delinquent is 3.99.

The 5-year average annual injury rate for delinquent mines is 27% higher than the national injury rate.
The 5-year average annual injury rate for delinquent mines is 34% higher than the rate for mines that are not currently delinquent.

### Segmenting Injury Rates

We probably want to take a look at the different types of mines. Maybe the reason for our delinquent mines being higher is because they are mostly coal mine and maybe coal mines have higher injury rates overall. Or maybe it's because they are mostly underground mines and those have a higher rate.

Let's get a feel for the composition of our delinquent mines and for the different injury rate segmentations we can do.

- Coal mines
    - Underground
    - Surface
    - Facility
- Metal/non-metal
    - Underground
    - Surface
    - Facility

In [85]:
############# All mines

coal_mines = mine_data[mine_data['c_m_ind'] == 'C']
metal_mines = mine_data[mine_data['c_m_ind'] == 'M']

active_mines = mine_data[mine_data['curr_stat_cd'] == 'Active']
active_coal_mines = coal_mines[coal_mines['curr_stat_cd'] == 'Active']
active_metal_mines = metal_mines[metal_mines['curr_stat_cd'] == 'Active']

active_ug_coal_mines = active_coal_mines[active_coal_mines['mine_type_cd'] == 'Underground']
active_ug_metal_mines = active_metal_mines[active_metal_mines['mine_type_cd'] == 'Underground']

active_s_coal_mines = active_coal_mines[active_coal_mines['mine_type_cd'] == 'Surface']
active_s_metal_mines = active_metal_mines[active_metal_mines['mine_type_cd'] == 'Surface']

active_fac_coal_mines = active_coal_mines[active_coal_mines['mine_type_cd'] == 'Facility']
active_fac_metal_mines = active_metal_mines[active_metal_mines['mine_type_cd'] == 'Facility']

**General stats about mines:**
- There are 40,176 mines in the U.S.
- There are 9,585 coal mines. 24% of all mines.
- There are 30,591 metal/non-metal mines. 76% of all mines.
- There are 6,269 active mines. 16% of all mines.

Coal
- There are 895 active coal mines. 14% of all active mines.
- There are 210 active, underground coal mines. 23% of all active coal mines.
- There are 426 active, surface coal mines. 48% of all active coal mines.
- There are 259 active, facility coal mines. 29% of all active coal mines.

Metal/Non-metal
- There are 5,374 active metal mines. 86% of all active mines.
- There are 197 active, underground metal mines. 4% of all active metal mines.
- There are 4,858 active, surface metal mines. 90% of all active metal mines.
- There are 318 active, metal facility mines. 6% of all active metal mines.

*In summary:*

Most of the mines in the U.S. are metal/non-metal mines. And really most of the active mines are metal/non-metal. Most mines are surface level and the split between underground and facility is about even.

In [84]:
########### Delinquent

del_keep = delinquency_data.rename(index=str, columns={'curr_cntrl_id':'del_curr_ctrl_id', 
                                                       'curr_cntrl_nm':'del_curr_ctrl_nm',
                                                       'curr_oprtr_id':'del_curr_oprtr_id',
                                                       'curr_oprtr_nm':'del_curr_oprtr_nm'})
del_detail = combine_del_mine_data(del_keep)

del_active_mines = del_detail[del_detail['curr_stat_cd'] == 'Active']

del_coal_mines = del_detail[del_detail['c_m_ind'] == 'C']
del_active_coal_mines = del_coal_mines[del_coal_mines['curr_stat_cd'] == 'Active']
del_active_ug_coal_mines = del_active_coal_mines[del_active_coal_mines['mine_type_cd'] == 'Underground']
del_active_s_coal_mines = del_active_coal_mines[del_active_coal_mines['mine_type_cd'] == 'Surface']
del_active_fac_coal_mines = del_active_coal_mines[del_active_coal_mines['mine_type_cd'] == 'Facility']

del_metal_mines = del_detail[del_detail['c_m_ind'] == 'M']
del_active_metal_mines = del_metal_mines[del_metal_mines['curr_stat_cd'] == 'Active']
del_active_ug_metal_mines = del_active_metal_mines[del_active_metal_mines['mine_type_cd'] == 'Underground']
del_active_s_metal_mines = del_active_metal_mines[del_active_metal_mines['mine_type_cd'] == 'Surface']
del_active_fac_metal_mines = del_active_metal_mines[del_active_metal_mines['mine_type_cd'] == 'Facility']

print(del_active_metal_mines['mine_id'].nunique())
print(del_active_ug_metal_mines['mine_id'].nunique())
print(del_active_s_metal_mines['mine_id'].nunique())
print(del_active_fac_metal_mines['mine_id'].nunique())

431
20
373
38


**General stats about delinquent mines:**
- There are 3,091 currently delinquent mines.
- 1,060 (34%) are coal mines
- 2,028 (66%) are metal/non-metal mines
- 638 (21%) are active mines

Active, delinquent coal
- There are 207 active, delinquent coal mines. 7% of all delinquent mines.
- 72 (35%) are underground mines
- 82 (40%) are surface mines
- 53 (26%) are facility mines

Active, delinquent metal/non-metal
- There are 431 active, delinquent metal mines. 14% of all active mines.
- 20 (5%) are underground mines
- 373 (87%) are surface mines
- 38 (9%) are facility mines

*In summary:*

There are twice as many active, metal/non-metal delinquent mines as active, coal delinquent mines but nearly 6 times more active, metal/non-metal mines than active, coal mines in general. This tells me that coal mines are more likely to get delinquencies than metal mines. This also tells me that, if injury rates are generally higher for coal mines, we can expect that our delinquent mines will have a higher injury rate because they have a larger composition of coal mines than the national composition.

So now I know I want to do injury rates for these subsets of the delinquency data:
- currently active coal
- currently active, underground coal
- currently active, surface coal
- currently active, facility coal
- currently active metal
- currently active, underground metal
- currently active, surface metal
- currently active, facility metal

But I'm going to take the currently active underground metal and facility metal mines with a grain of salt because they are so few in number and do not compose much of the delinquency data.

I'm going to compare each to the national and non-delinquent equivalent as well and see where we stand.

And I also wonder about limiting this to only active mines. Why would we do that? Our equation already only calculates rates for mines with over 2,000 employee hours per year. And we're going to want to figure out how many of these delinquent mines have been delinquent since 2014... that might cut our numbers down too far.



In [139]:
print(list(range(2014,2019,1)))

[2014, 2015, 2016, 2017, 2018]


In [167]:
def filter_mines(c_m, mine_type, mine_status, del_status, hr_text):
    if c_m:
        filter_cm = mine_data[mine_data['c_m_ind'] == c_m]['mine_id'].unique()
    else:
        filter_cm = mine_data['mine_id'].unique()
    if mine_type:
        filter_mine_type = mine_data[mine_data['mine_type_cd'] == mine_type]['mine_id'].unique()
    else:
        filter_mine_type = mine_data['mine_id'].unique()
    if mine_status:
        filter_mine_status = mine_data[mine_data['curr_stat_cd'] == mine_status]['mine_id'].unique()
    else:
        filter_mine_status = mine_data['mine_id'].unique()
    
    injuries = inj_data[(inj_data['mine_id'].isin(filter_cm)) & (inj_data['mine_id'].isin(filter_mine_type)) & (inj_data['mine_id'].isin(filter_mine_status))]
    hours = hrs_data[(hrs_data['mine_id'].isin(filter_cm)) & (hrs_data['mine_id'].isin(filter_mine_type)) & (hrs_data['mine_id'].isin(filter_mine_status))]
    
    if del_status == 'Delinquent':
        injuries = injuries[injuries['mine_id'].isin(get_delinquent_since(2014))]
        hours = hours[hours['mine_id'].isin(get_delinquent_since(2014))]
    elif del_status == 'Nondelinquent':
        injuries = injuries[~injuries['mine_id'].isin(get_delinquent_mines())]
        hours = hours[~hours['mine_id'].isin(get_delinquent_mines())]
    elif del_status == 'All':
        injuries = injuries
        hours = hours
    
    #10-year average injury rate
    #print(get_range_injury_rate(injuries, hours, hr_text, list(range(2009,2019,1))))
    #5-year average injury rate
    #print(get_range_injury_rate(injuries, hours, hr_text, list(range(2014,2019,1))))
    #ALL-year average injury rate
    print(get_range_injury_rate(injuries, hours, hr_text, list(range(1994,2019,1))))
    

#filter_mines('', '', '', '', 'All mines')
#filter_mines('', '', '', 'Delinquent', 'All delinquent since 2014')
#filter_mines('', '', '', 'Nondelinquent', 'All nondelinquent')
print('*****Avg Injury Rates For All Mines*****')
filter_mines('', '', '', '', 'All mines')
filter_mines('', '', '', 'Delinquent', 'All delinquent mines')
filter_mines('', '', '', 'Nondelinquent', 'All nondelinquent mines')
filter_mines('', '', 'Active', '', 'Active mines')
filter_mines('', '', 'Active', 'Delinquent', 'Active, delinquent mines')
filter_mines('', '', 'Active', 'Nondelinquent', 'Active, nondelinquent mines')
print('')
print('*****Avg Injury Rates For Coal Mines*****')
filter_mines('C', '', '', '', 'All coal mines')
filter_mines('C', '', '', 'Delinquent', 'All delinquent coal mines')
filter_mines('C', '', '', 'Nondelinquent', 'All nondelinquent coal mines')
filter_mines('C', '', 'Active', '', 'Active coal mines')
filter_mines('C', '', 'Active', 'Delinquent', 'Active, delinquent coal mines')
filter_mines('C', '', 'Active', 'Nondelinquent', 'Active, nondelinquent coal mines')
print('')
filter_mines('C', 'Underground', '', '', 'All underground coal mines')
filter_mines('C', 'Underground', '', 'Delinquent', 'All delinquent, underground coal mines')
filter_mines('C', 'Underground', '', 'Nondelinquent', 'All nondelinquent, underground coal mines')
filter_mines('C', 'Underground', 'Active', '', 'Active underground coal mines')
filter_mines('C', 'Underground', 'Active', 'Delinquent', 'Active, delinquent, underground coal mines')
filter_mines('C', 'Underground', 'Active', 'Nondelinquent', 'Active, nondelinquent, underground coal mines')
print('')
filter_mines('C', 'Surface', '', '', 'All surface coal mines')
filter_mines('C', 'Surface', '', 'Delinquent', 'All delinquent, surface coal mines')
filter_mines('C', 'Surface', '', 'Nondelinquent', 'All nondelinquent, surface coal mines')
filter_mines('C', 'Surface', 'Active', '', 'Active surface coal mines')
filter_mines('C', 'Surface', 'Active', 'Delinquent', 'Active, delinquent, surface coal mines')
filter_mines('C', 'Surface', 'Active', 'Nondelinquent', 'Active, nondelinquent, surface coal mines')
print('')
filter_mines('C', 'Facility', '', '', 'All facility coal mines')
filter_mines('C', 'Facility', '', 'Delinquent', 'All delinquent, facility coal mines')
filter_mines('C', 'Facility', '', 'Nondelinquent', 'All nondelinquent, facility coal mines')
filter_mines('C', 'Facility', 'Active', '', 'Active facility coal mines')
filter_mines('C', 'Facility', 'Active', 'Delinquent', 'Active, delinquent, facility coal mines')
filter_mines('C', 'Facility', 'Active', 'Nondelinquent', 'Active, nondelinquent, facility coal mines')
print('')
print('*****Avg Injury Rates For Metal Mines*****')
filter_mines('M', '', '', '', 'All metal/nm mines')
filter_mines('M', '', '', 'Delinquent', 'All delinquent metal/nm mines')
filter_mines('M', '', 'Active', '', 'Active metal/nm mines')
filter_mines('M', '', 'Active', 'Delinquent', 'Active, delinquent metal/nm mines')
print('')
filter_mines('M', 'Underground', '', '', 'All underground metal/nm mines')
filter_mines('M', 'Underground', '', 'Delinquent', 'All delinquent, underground metal/nm mines')
filter_mines('M', 'Underground', 'Active', '', 'Active underground metal/nm mines')
filter_mines('M', 'Underground', 'Active', 'Delinquent', 'Active, delinquent, underground metal/nm mines')
print('')
filter_mines('M', 'Surface', '', '', 'All surface metal/nm mines')
filter_mines('M', 'Surface', '', 'Delinquent', 'All delinquent, surface metal/nm mines')
filter_mines('M', 'Surface', 'Active', '', 'Active surface metal/nm mines')
filter_mines('M', 'Surface', 'Active', 'Delinquent', 'Active, delinquent, surface metal/nm mines')
print('')
filter_mines('M', 'Facility', '', '', 'All facility metal/nm mines')
filter_mines('M', 'Facility', '', 'Delinquent', 'All delinquent, facility metal/nm mines')
filter_mines('M', 'Facility', 'Active', '', 'Active facility metal/nm mines')
filter_mines('M', 'Facility', 'Active', 'Delinquent', 'Active, delinquent, facility metal/nm mines')

*****Avg Injury Rates For All Mines*****
All mines: 4.32
All delinquent mines: 5.56
All nondelinquent mines: 4.15
Active mines: 3.86
Active, delinquent mines: 4.3
Active, nondelinquent mines: 3.72

*****Avg Injury Rates For Coal Mines*****
All coal mines: 5.72
All delinquent coal mines: 7.05
All nondelinquent coal mines: 5.4
Active coal mines: 4.62
Active, delinquent coal mines: 4.47
Active, nondelinquent coal mines: 4.25

All underground coal mines: 8.29
All delinquent, underground coal mines: 9.63
All nondelinquent, underground coal mines: 8.13
Active underground coal mines: 6.98
Active, delinquent, underground coal mines: 5.04
Active, nondelinquent, underground coal mines: 6.89

All surface coal mines: 2.85
All delinquent, surface coal mines: 4.42
All nondelinquent, surface coal mines: 2.73
Active surface coal mines: 2.31
Active, delinquent, surface coal mines: 3.52
Active, nondelinquent, surface coal mines: 2.15

All facility coal mines: 5.79
All delinquent, facility coal mines: 5.

## Adding Controller At Time Of Delinquency (CATOD)


In [64]:
#cntrlr_hist_data - ['oper_id', 'ctrlr_id', 'ctrlr_start_dt', 'ctrlr_nm', 'ctrlr_end_dt',
#                    'oper_nm', 'load_dt']
#delinquency_data - ['Unnamed: 0', 'Assess Case Nbr', 'Mine ID', 'Mine ID.1', 'Mine Status',
#       'Violator ID', 'Violator Name', 'Violator Type', 'Controller Id',
#       'Controller Name', 'Age Category', 'Delinquent Date', 'Delinquent Type',
#       'Ending Balance', 'Current Operator ID', 'Current Operator Name',
#       'year', 'month', 'yearmonth']

#What this merge means: cntrlr_hist_data is going to look in delinquency_data and try to find matches on the
#Violator ID column. If it finds more than one match, it will duplicate itself to accommodate all of the rows
#in delinquency_data. And we know it will find more than one matching row in delinquency_data because Violators
#in there multiple times for multiple offense at different times. But this is ok because in the next step
#we're going to filter out all of the cases where the dates don't match. This will probably still leave us with
#more records than we want, but that's where we start dropping dups and joining things back onto the OG delinquency_data
catod_del_merge = pd.merge(cntrlr_hist_data,delinquency_data,how='left',left_on='oper_id',right_on='violator_id')

#Filters for the filtering we'll be doing on catod_del_merge
gt_dates = catod_del_merge['delinquent_date'] >= catod_del_merge['ctrlr_start_dt']
lt_dates = catod_del_merge['delinquent_date'] <= catod_del_merge['ctrlr_end_dt']

#So here's where we drop the records where the dates don't match up
catod_filtered = catod_del_merge[(gt_dates) & (lt_dates)]
print(catod_filtered.columns)
#We're getting rid of columns so that we can drop duplicates and get back to our original
#delinquency_data count
catod_reduced = catod_filtered[['oper_id','ctrlr_id','ctrlr_start_dt','ctrlr_end_dt','oper_nm',
                                'ass_case_nbr','curr_cntrl_id','mine_id','delinquent_date',
                                'ending_balance']].drop_duplicates()

catod_reduced = catod_reduced.rename(index=str, columns={'oper_id':'catod_oper_id',
                                                         'ctrlr_id':'catod_ctrlr_id',
                                                         'ctrlr_start_dt':'catod_ctrlr_start_dt',
                                                         'ctrlr_end_dt':'catod_ctrlr_end_dt',
                                                         'oper_nm':'catod_oper_nm'})

Index(['oper_id', 'ctrlr_id', 'ctrlr_start_dt', 'ctrlr_nm', 'ctrlr_end_dt',
       'oper_nm', 'load_dt', 'ass_case_nbr', 'mine_id', 'mine_status',
       'violator_id', 'violator_nm', 'violator_type', 'curr_cntrl_id',
       'curr_cntrl_nm', 'age_cateogry', 'delinquent_date', 'delinquent_type',
       'ending_balance', 'curr_oprtr_id', 'curr_oprtr_nm', 'year', 'month'],
      dtype='object')


### Testing CATOD

In [68]:
#At this point I ran a little test to see how we're doing and it looks like we're close but
#there are a few records missing in the catod_reduced dataframe.
print(len(delinquency_data))
print(len(catod_reduced))
print(delinquency_data['ending_balance'].sum())
print(catod_reduced['ending_balance'].sum())
print(catod_reduced.columns)

13052
13048
45098251.75
45096602.55
Index(['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr', 'curr_cntrl_id',
       'mine_id', 'delinquent_date', 'ending_balance'],
      dtype='object')


In [66]:
#I'm going to analyze those two dataframes side by side to see what's up.
catod_by_mine = catod_reduced.groupby('mine_id')['ass_case_nbr'].count().reset_index()
del_by_mine = delinquency_data.groupby('mine_id')['ass_case_nbr'].count().reset_index()

del_compare = pd.merge(del_by_mine, catod_by_mine, how='left', on='mine_id', suffixes=('_del','_catod'))
del_compare['diff'] = del_compare.apply(lambda row: 1 if row['ass_case_nbr_del'] != row['ass_case_nbr_catod'] else 0, axis=1)

print(del_compare.sort_values('diff', ascending=False))

      mine_id  ass_case_nbr_del  ass_case_nbr_catod  diff
1064  1519775                17                18.0     1
2579  4406787                 1                 NaN     1
2555  4404481                 1                 NaN     1
2784  4608137                 1                 NaN     1
26     103102                 9                 7.0     1
2062  3609989                 3                 3.0     0
2055  3609930                 1                 1.0     0
2056  3609943                 1                 1.0     0
2057  3609944                 1                 1.0     0
2058  3609950                 5                 5.0     0
2059  3609958                 8                 8.0     0
2060  3609963                25                25.0     0
2061  3609970                 3                 3.0     0
2064  3609991                 2                 2.0     0
2063  3609990                 1                 1.0     0
2065  3610001                 3                 3.0     0
2066  3610008 

In [ ]:
#Now I can see which records we've got issues with between our original delinquency_data and our new
#catod delinquency data: ['1519775','4406787','4404481','4608137','103102']
#So let's checkout out what the differences are:
del_keep_cols = ['ass_case_nbr','mine_id','delinquent_date','ending_balance','violator_id']
catod_keep_cols = ['ass_case_nbr','mine_id','delinquent_dt','ending_balance',
                  'catod_ctrlr_id', 'catod_ctrlr_start_dt', 'catod_ctrlr_end_dt', 'catod_oper_nm']

In [ ]:
#I can see that this is the record with the overlapping controllers.
print(delinquency_data[del_keep_cols][delinquency_data['mine_id'] == 1519775].sort_values('ending_balance'))
print(catod_reduced[catod_keep_cols][catod_reduced['mine_id'] == 1519775].sort_values('ending_balance'))
#print(catod_reduced.columns)

In [ ]:
#Mine ID = 4406787 seems to be in my delinquency_data but not my catod data. I think the reason for this
#is that there was either no matching Violator ID in the cntrlr_hist_data or there was a matching ID but
#no matching controller dates.
print(delinquency_data[del_keep_cols][delinquency_data['mine_id'] == 4406787].sort_values('ending_balance'))
print(catod_reduced[catod_keep_cols][catod_reduced['mine_id'] == 4406787].sort_values('ending_balance'))

#And my latter assersion is correct. P22558 is in cntrlr_hist_data but the end date of the controller
#is before the Delinquency Date of the delinquency_data record
print(cntrlr_hist_data[cntrlr_hist_data['oper_id']=='P22558'])

In [ ]:
#Again, it appears to be a matching issue between delinquency_data and cntrlr_hist_data
print(delinquency_data[del_keep_cols][delinquency_data['mine_id'] == 4404481].sort_values('ending_balance'))
print(catod_reduced[catod_keep_cols][catod_reduced['mine_id'] == 4404481].sort_values('ending_balance'))

#Same issue as before. controller end date is before Delinquency date
print(cntrlr_hist_data[cntrlr_hist_data['oper_id']=='P23566'])

In [ ]:
#Probably the same here
print(delinquency_data[del_keep_cols][delinquency_data['mine_id'] == 4608137].sort_values('ending_balance'))
print(catod_reduced[catod_keep_cols][catod_reduced['mine_id'] == 4608137].sort_values('ending_balance'))

#Same issue as before. controller end date is before Delinquency date
print(cntrlr_hist_data[cntrlr_hist_data['oper_id']=='P23546'])

In [ ]:
#And this one seems to be missing a few records from the catod_reduced that are in the delinquency_data.
#I'm going to guess that it is the same as before
print(delinquency_data[del_keep_cols][delinquency_data['mine_id'] == 103102].sort_values('ending_balance'))
print(catod_reduced[catod_keep_cols][catod_reduced['mine_id'] == 103102].sort_values('ending_balance'))

#Same issue as before. controller end date is before Delinquency date
print(cntrlr_hist_data[cntrlr_hist_data['oper_id']=='P23732'])

Here's another test that we'll do just on the Justice files:

In [ ]:
#I tested out a bunch of the catod operators from the Justice file with the script below
#and everything seems good
print(cntrlr_hist_data[['oper_id','ctrlr_id','ctrlr_start_dt','ctrlr_end_dt','oper_nm']]
      [cntrlr_hist_data['oper_id']=='P21141'])

## Regional Debtors

In [80]:
#Debt That They are Currently Responsible For
def get_responsible_debt(ctrlr_id):
    return catod_reduced[catod_reduced['curr_cntrl_id'].isin(ctrlr_id)]['ending_balance'].sum()


#Mine Count from Responsible Debt
def get_responsible_records(ctrlr_id):
    return catod_reduced[catod_reduced['curr_cntrl_id'].isin(ctrlr_id)]['mine_id'].count()

def get_responsible_mines(ctrlr_id):
    return catod_reduced[catod_reduced['curr_cntrl_id'].isin(ctrlr_id)]['mine_id'].nunique()


#Debt That They Originated
def get_originated_debt(ctrlr_id):
    return catod_reduced[catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)]['ending_balance'].sum()


#Mine Count from Originated Debt
def get_originated_records(ctrlr_id):
    return catod_reduced[catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)]['mine_id'].count()

def get_originated_mines(ctrlr_id):
    return catod_reduced[catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)]['mine_id'].nunique()


#Debt That They Originated And Are Still Responsible For
def get_originated_responsible_debt(ctrlr_id):
    originated = catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = catod_reduced['curr_cntrl_id'].isin(ctrlr_id)
    return catod_reduced[(originated) & (responsible)]['ending_balance'].sum()


#Debt That They Originated But Are NOT Still Responsible For
def get_originated_not_responsible_debt(ctrlr_id):
    originated = catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = catod_reduced['curr_cntrl_id'].isin(ctrlr_id)
    return catod_reduced[(originated) & (~responsible)]['ending_balance'].sum()


#Debt That They Are Responsible For But Did Not Originate
def get_not_originated_responsible_debt(ctrlr_id):
    originated = catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = catod_reduced['curr_cntrl_id'].isin(ctrlr_id)
    return catod_reduced[(~originated) & (responsible)]['ending_balance'].sum()


#Output the delinquent debt as a csv
def print_responsible_debt(ctrlr_id, ctrlr_nm):
    responsible_data = catod_reduced[catod_reduced['curr_cntrl_id'].isin(ctrlr_id)]
    responsible_data.to_csv(data_dir + 'analysis/by_controller/' + slugify(ctrlr_nm) + '-responsible-delinquencies.csv')
    
def print_originated_debt(ctrlr_id, ctrlr_nm):
    originated_data = catod_reduced[catod_reduced['catod_ctrlr_id'].isin(ctrlr_id)]
    originated_data.to_csv(data_dir + 'analysis/by_controller/' + slugify(ctrlr_nm) + '-originated-delinquencies.csv')


regional_debtors = [{'cntrl_nm':'James C Justice II', 'cntrl_id':['C04355']},
                    {'cntrl_nm':'James C  Justice lll; Jillian L  Justice', 'cntrl_id':['0091855']},
                    {'cntrl_nm':'Justice family companies', 'cntrl_id':['0091855','C04355']},
                    {'cntrl_nm':'Jeffrey Hoops, Revelation Energy', 'cntrl_id':['C15863']},
                    {'cntrl_nm':'Virginia Conservation Legacy Fund/ERP Compliant','cntrl_id':['0121101']},
                    {'cntrl_nm':'James H Booth','cntrl_id':['C11194']}]

total_del_sum = catod_reduced['ending_balance'].sum()

for cntrl in regional_debtors:
    ctrlr_id = cntrl['cntrl_id']
    ctrlr_nm = cntrl['cntrl_nm']
    responsible = '${:,.2f}'.format(get_responsible_debt(ctrlr_id))
    originated = '${:,.2f}'.format(get_originated_debt(ctrlr_id))
    originated_responsible = '${:,.2f}'.format(get_originated_responsible_debt(ctrlr_id))
    originated_not_responsible = '${:,.2f}'.format(get_originated_not_responsible_debt(ctrlr_id))
    not_originated_responsible = '${:,.2f}'.format(get_not_originated_responsible_debt(ctrlr_id))
    
    responsible_mines = get_responsible_mines(ctrlr_id)
    originated_mines = get_originated_mines(ctrlr_id)
    
    responsible_records = get_responsible_records(ctrlr_id)
    originated_records = get_originated_records(ctrlr_id)
    
    print_responsible_debt(ctrlr_id, ctrlr_nm)
    print_originated_debt(ctrlr_id, ctrlr_nm)
    
    print('------------------')
    print(ctrlr_nm)
    print('------------------')
    print('Responsible: ' + responsible)
    print('Originated: ' + originated)
    print('Originated & Responsible: ' + originated_responsible)
    print('Originated but NOT Responsible: ' + originated_not_responsible)
    print('Responsible but NOT Originated: ' + not_originated_responsible)
    print('')
    print('Unique mines with responsible debt: ' + str(responsible_mines))
    print('Unique mines with originated debt: ' + str(originated_mines))
    print('')
    print('Responsible debt records: ' + str(responsible_records))
    print('Originated debt records: ' + str(originated_records))
    print('')
    
    print('Details for controller: ' + ctrlr_nm + ' (' + '; '.join(ctrlr_id) + ')')
    print('When you were controller at ' + str(originated_mines) + ' mines you originated delinquent debt that now totals ' 
          + originated + '.')
    print('Some of those mines you still own, some have been sold. Our understanding is that debt remains with the mines.')
    print('According to our analysis of MSHA data, you currently control ' + str(responsible_mines) 
          + ' mines and are responsible for ' + responsible + ' in delinquent debt at those mines.')
    print('')
    
    
    


------------------
James C Justice II
------------------
Responsible: $518,182.42
Originated: $4,790,373.26
Originated & Responsible: $518,182.42
Originated but NOT Responsible: $4,272,190.84
Responsible but NOT Originated: $0.00

Unique mines with responsible debt: 11
Unique mines with originated debt: 74

Responsible debt records: 83
Originated debt records: 588

Details for controller: James C Justice II (C04355)
When you were controller at 74 mines you originated delinquent debt that now totals $4,790,373.26.
Some of those mines you still own, some have been sold. Our understanding is that debt remains with the mines.
According to our analysis of MSHA data, you currently control 11 mines and are responsible for $518,182.42 in delinquent debt at those mines.

------------------
James C  Justice lll; Jillian L  Justice
------------------
Responsible: $3,785,194.85
Originated: $857,058.60
Originated & Responsible: $761,183.07
Originated but NOT Responsible: $95,875.53
Responsible but 

I want Justice Sr. originated, delinquent mines in WV count. UNIQUE COUNT.

In [82]:
justice_sr_originated_mines = catod_full[catod_full['catod_ctrlr_id'].isin(['C04355'])]
justice_sr_originated_mines_wv = justice_sr_originated_mines[justice_sr_originated_mines['state_abbr'] == 'WV']
print(justice_sr_originated_mines['mine_id'].nunique())
print(justice_sr_originated_mines_wv['mine_id'].nunique())

74
12


## General Delinquency Facts

In [75]:
print(catod_full.columns)

Index(['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr', 'curr_cntrl_id',
       'mine_id', 'delinquent_date', 'ending_balance', 'curr_mine_nm',
       'c_m_ind', 'mine_type_cd', 'curr_stat_cd', 'curr_stat_dt',
       'curr_cntrlr_id', 'curr_cntrlr_nm', 'curr_oper_id', 'curr_oper_nm',
       'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt', 'latest_del_date',
       'earliest_del_date', 'total_due', 'num_del_records', 'num_cntrlrs',
       'cntrlrs_concat'],
      dtype='object')


Here we want to join our `catod_reduced` data with general mine data so we can do some queries.

In [74]:
catod_full = combine_del_mine_data(catod_reduced)
#Mines with delinquent debt: 3,088
#Active mines with DD: 638 (10% of all active mines)
#Coal mines with DD: 1,057 (11% of all coal mines)
#Active, coal mines with DD: 207 (23%)

total_due = catod_full['ending_balance'].sum()
mines_w_debt = catod_full['mine_id'].nunique()
active_mines_w_debt = catod_full[(catod_full['curr_stat_cd'] == 'Active')]['mine_id'].nunique()
coal_mines_w_debt = catod_full[(catod_full['c_m_ind'] == 'C')]['mine_id'].nunique()
active_coal_mines_w_debt = catod_full[catod_full['curr_stat_cd'] == 'Active'].groupby('c_m_ind')['mine_id'].nunique()['C']

mines = mine_data['mine_id'].nunique()
active_mines = mine_data[(mine_data['curr_stat_cd'] == 'Active')]['mine_id'].nunique()
coal_mines = mine_data[(mine_data['c_m_ind'] == 'C')]['mine_id'].nunique()
active_coal_mines = mine_data[mine_data['curr_stat_cd'] == 'Active'].groupby('c_m_ind')['mine_id'].nunique()['C'] 

#print('Total delinquent debt: ' + '${:,.2f}'.format(total_due))
print('Unique mines with debt: ' + '{:,.0f}'.format(mines_w_debt) + ' ({:,.2f}'.format((mines_w_debt/mines)*100) + '%)')
print('Active mines with debt: ' + '{:,.0f}'.format(active_mines_w_debt) + ' ({:,.2f}'.format((active_mines_w_debt/active_mines)*100) + '%)')
print('Coal mines with debt: ' + '{:,.0f}'.format(coal_mines_w_debt) + ' ({:,.2f}'.format((coal_mines_w_debt/coal_mines)*100) + '%)')
print('Active coal mines with debt: ' + '{:,.0f}'.format(active_coal_mines_w_debt) + ' ({:,.2f}'.format((active_coal_mines_w_debt/active_coal_mines)*100) + '%)')
print('')
print('Unique mines: ' + '{:,.0f}'.format(mines))
print('Active mines: ' + '{:,.0f}'.format(active_mines))
print('Coal mines: ' + '{:,.0f}'.format(coal_mines))
print('Active coal mines: ' + '{:,.0f}'.format(active_coal_mines))


#del_5yr = catod_full[catod_full['earliest_del_date'] < pd.to_datetime('2013-12-05', format='%Y-%m-%d')]
#del_mines_by_type = del_5yr[del_5yr['curr_stat_cd'] == 'Active'].groupby('c_m_ind')['mine_id'].nunique()
#mines_by_type = mine_data[mine_data['curr_stat_cd'] == 'Active'].groupby('c_m_ind')['mine_id'].nunique()
#print(del_mines_by_type)
#print(mines_by_type)


Unique mines with debt: 3,088 (7.69%)
Active mines with debt: 638 (10.18%)
Coal mines with debt: 1,057 (11.03%)
Active coal mines with debt: 207 (23.13%)

Unique mines: 40,176
Active mines: 6,269
Coal mines: 9,585
Active coal mines: 895
